In [1]:
import pandas as pd
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [2]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [3]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [4]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
dependent=dataset[['charges']]

In [5]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [6]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
#n_estimators-->int, default=100
#learning_rate: float, default=0.1
#loss{‘squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’}, default=’squared_error’
#criterion{‘friedman_mse’, ‘squared_error’}, default=’friedman_mse’
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
param_grid={'n_estimators':[10,50,80,100],"learning_rate":[0.3,0.1,1.0,1.5,2],"loss":['squared_error', 'absolute_error', 'huber', 'quantile'],
            "criterion":['squared_error', 'friedman_mse']}
grid=GridSearchCV(GradientBoostingRegressor(),param_grid,refit=True,n_jobs=-1,verbose=3)
grid.fit(Xtrain,Ytrain)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


D:\Anaconda\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'learning_rate': [0.3, 0.1, 1.0, 1.5, 2],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'n_estimators': [10, 50, 80, 100]},
             verbose=3)

In [7]:
print(grid.best_params_)

{'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'huber', 'n_estimators': 50}


In [8]:
table=pd.DataFrame.from_dict(grid.cv_results_)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_learning_rate,param_loss,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029408,0.004324,0.003912,0.000102,squared_error,0.3,squared_error,10,"{'criterion': 'squared_error', 'learning_rate'...",0.828587,0.805939,0.870475,0.849395,0.793642,0.829607,0.027987,8
1,0.081394,0.003865,0.003925,0.000535,squared_error,0.3,squared_error,50,"{'criterion': 'squared_error', 'learning_rate'...",0.815767,0.757252,0.854879,0.821947,0.770011,0.803971,0.035749,26
2,0.121062,0.005391,0.003828,0.000413,squared_error,0.3,squared_error,80,"{'criterion': 'squared_error', 'learning_rate'...",0.804663,0.738866,0.846963,0.803393,0.752458,0.789269,0.039142,38
3,0.135313,0.003209,0.003717,0.000513,squared_error,0.3,squared_error,100,"{'criterion': 'squared_error', 'learning_rate'...",0.797243,0.738007,0.837870,0.794738,0.740380,0.781648,0.037903,41
4,0.044967,0.005883,0.003344,0.000907,squared_error,0.3,absolute_error,10,"{'criterion': 'squared_error', 'learning_rate'...",0.721557,0.545458,0.743213,0.739836,0.620431,0.674099,0.078459,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.281834,0.007213,0.009354,0.001537,friedman_mse,2.0,huber,100,"{'criterion': 'friedman_mse', 'learning_rate':...",-0.205027,-0.016253,-0.481565,-0.102358,-0.088520,-0.178744,0.162954,143
156,0.116681,0.005037,0.009169,0.002876,friedman_mse,2.0,quantile,10,"{'criterion': 'friedman_mse', 'learning_rate':...",-3.337165,-3.673905,-2.849714,-4.419637,-2.274547,-3.310994,0.727802,154
157,0.499566,0.003710,0.008433,0.004253,friedman_mse,2.0,quantile,50,"{'criterion': 'friedman_mse', 'learning_rate':...",-3.269344,-3.023913,-5.894840,-4.959063,-2.564026,-3.942237,1.268240,157
158,0.693353,0.036641,0.006100,0.003392,friedman_mse,2.0,quantile,80,"{'criterion': 'friedman_mse', 'learning_rate':...",-3.168079,-2.954218,-4.147498,-5.369655,-2.715268,-3.670943,0.979041,156


In [9]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 21
BMI: 25.8
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [10]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
#␣change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[2155.63544263]


D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
